In [15]:
import pandas as pd
from Bio.Cluster import kcluster
from collections import Counter
import numpy as np
import subprocess
import glob
import os

In [4]:
### Analysis input
input="XBB15-LCB1"
input_dataframe=pd.read_csv(f"output/{input}/opt_binders/filtered_binders.csv")
input_dataframe.describe()

,Unnamed: 0,score,plddt,i_ptm,i_pae,rmsd,ddg,rg,charge,sap,dG,dSASA,dG_dSASA_ratio,int_unsat_hbonds,int_hbonds,hyd_contacts,shape_comp,ddg_score,ddg_dsasa_100,ddgscore_dsasa_100
count,164.00000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000
mean,81.50000,0.826036,0.955420,0.901549,3.943733,0.146916,-51.231841,10.040948,-3.655313,25.629001,67.423314,1650.549033,3.949831,10.810976,6.743902,7.036585,0.678313,-36.422164,-3.106970,-2.221277
std,47.48684,0.039941,0.000811,0.005787,0.067221,0.051442,2.054586,0.030379,1.058477,5.482801,69.000298,82.246273,4.098190,3.946840,1.592243,0.244955,0.015785,5.087742,0.099030,0.389938
min,0.00000,0.730676,0.954476,0.888785,3.822869,0.052879,-54.357000,9.960623,-6.800058,17.582819,-23.196578,1231.566345,-1.519539,3.000000,1.000000,7.000000,0.637880,-49.238138,-3.672559,-3.140232
25%,40.75000,0.798263,0.954810,0.895682,3.910507,0.110071,-52.964750,10.014651,-4.757677,20.690872,-3.988860,1591.412872,-0.246005,8.000000,5.000000,7.000000,0.668589,-39.770990,-3.126144,-2.497758
50%,81.50000,0.824682,0.955170,0.904840,3.944346,0.139946,-52.199500,10.031329,-3.701676,23.650586,78.631304,1696.164010,4.596325,12.000000,7.000000,7.000000,0.676787,-35.319272,-3.089311,-2.088624
75%,122.25000,0.855757,0.955790,0.906909,3.966463,0.178190,-49.412000,10.072030,-2.767276,30.991885,116.188504,1716.538268,6.771054,14.000000,8.000000,7.000000,0.683489,-32.830431,-3.050420,-1.926683
max,163.00000,0.950638,0.957900,0.911125,4.429562,0.341782,-45.230000,10.089997,-2.682179,36.945316,304.327951,1745.231987,18.433581,19.000000,9.000000,9.000000,0.726146,-23.811998,-3.000509,-1.396189


## Filter dataframe
tipical metrics:
- plddt > 0.85-0.9
- i_pae < 5
- ddg_dsasa_100 < -2.5 - -3.5 (sort by)
additional metrics:
- charge < -2
- hyd_contacts > 4-6
- sap < 30-60

In [10]:
filtered = input_dataframe[(input_dataframe["plddt"]>0.90)&
                           (input_dataframe["ddg"]<-30)&
                           (input_dataframe["charge"]<-2)&
                           (input_dataframe["ddg_dsasa_100"]<-2.5)&
                           (input_dataframe["shape_comp"]>0.62)&
                           (input_dataframe["hyd_contacts"]>6)&
                           (input_dataframe["sap"]<40)]#&
                           #(input_dataframe["dG"]<30)]

filtered = filtered.sort_values(by='ddg_dsasa_100', ascending=True).drop_duplicates("model_path")
filtered

,Unnamed: 0,score,plddt,i_ptm,i_pae,rmsd,model_path,input_pdb,ddg,rg,...,dSASA,dG_dSASA_ratio,int_unsat_hbonds,int_hbonds,hyd_contacts,shape_comp,ddg_score,ddg_dsasa_100,ddgscore_dsasa_100,seq
0,0,0.766746,0.955639,0.888785,4.376522,0.199316,output/XBB15-LCB1/opt_binders/binders/579_470/...,output/XBB15-LCB1/opt_binders/binders/579_470/...,-45.230,10.023927,...,1231.566345,1.525011,10.0,1.0,9.0,0.707727,-36.598774,-3.672559,-2.971726,SPFARAVEIHRRAMELYKEGKKEEANKVFELLREWAKTKDESLLDE...
1,1,0.785096,0.954575,0.889905,4.429562,0.216167,output/XBB15-LCB1/opt_binders/binders/579_470/...,output/XBB15-LCB1/opt_binders/binders/579_470/...,-46.007,10.036078,...,1258.483147,0.203190,11.0,1.0,9.0,0.726146,-36.819847,-3.655750,-2.925732,SPFARAVEIHKRAMELYKKGKKEEAEKVFELLREWAKTKNEELLDE...
2,2,0.812374,0.954649,0.894636,4.008572,0.086369,output/XBB15-LCB1/opt_binders/binders/740_279/...,output/XBB15-LCB1/opt_binders/binders/740_279/...,-52.574,10.067076,...,1540.575276,-0.521886,5.0,8.0,7.0,0.676264,-42.909874,-3.412621,-2.785315,SSLDRLNKIVAEAKELETLGKFEESLEVYDLIGKYLNTKDEKYLDE...
3,3,0.745213,0.955274,0.895376,3.957288,0.104024,output/XBB15-LCB1/opt_binders/binders/740_279/...,output/XBB15-LCB1/opt_binders/binders/740_279/...,-51.186,10.077020,...,1515.589685,-0.729657,4.0,9.0,7.0,0.681681,-43.755070,-3.377299,-2.887000,SDLDRLNKIVKEAKELETLGKYEESLEVYDLIGKYLNTKDKKYLDE...
4,4,0.795052,0.955477,0.895440,3.992614,0.106472,output/XBB15-LCB1/opt_binders/binders/740_279/...,output/XBB15-LCB1/opt_binders/binders/740_279/...,-51.736,10.078912,...,1546.347072,-1.320101,4.0,9.0,7.0,0.670714,-45.126033,-3.345691,-2.918234,SDLERLNKIVKEAKELETLGKFEESLEVYDLIGKYLNTKDKKYLDE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,159,0.797686,0.955196,0.895754,3.965950,0.086379,output/XBB15-LCB1/opt_binders/binders/740_279/...,output/XBB15-LCB1/opt_binders/binders/740_279/...,-47.658,10.077405,...,1585.979148,-0.494048,4.0,9.0,7.0,0.702933,-46.986712,-3.004958,-2.962631,SDLDRLNKIVKEAEELKTLGKYEESLDVYDLIGEYLNTKDKKYLDE...
160,160,0.826284,0.954879,0.899444,3.875973,0.211538,output/XBB15-LCB1/opt_binders/binders/976_304/...,output/XBB15-LCB1/opt_binders/binders/976_304/...,-48.283,10.034939,...,1607.590893,1.195900,18.0,5.0,7.0,0.675635,-40.154567,-3.003438,-2.497810,SSLERANRLVAEADALAAEGKFEAALDIKELVGLYFNTKDESYLKQ...
161,161,0.876326,0.955508,0.896881,3.956453,0.341782,output/XBB15-LCB1/opt_binders/binders/976_304/...,output/XBB15-LCB1/opt_binders/binders/976_304/...,-48.452,10.056201,...,1614.080291,0.655237,15.0,5.0,7.0,0.668648,-33.356205,-3.001833,-2.066577,SNLDRANRLVEEADKLEKQGKWEEALDIKELVGLYYNTKDESYLTK...
162,162,0.769324,0.955102,0.895309,3.941662,0.231736,output/XBB15-LCB1/opt_binders/binders/976_304/...,output/XBB15-LCB1/opt_binders/binders/976_304/...,-48.580,10.078514,...,1618.360753,0.130637,10.0,5.0,7.0,0.658425,-36.048491,-3.001803,-2.227469,SDLDKANELVKEAEELEKEGKYEEALEIKELVGLYFNTKDKSYLTK...


# Cluster sequences

In [66]:
num_clusters=10

matrix = np.asarray([np.fromstring(s, dtype=np.uint8) for s in filtered["seq"].to_list()])
#matrix = np.asarray([np.frombuffer(seq.encode(), dtype=np.uint8) for seq in filtered["seq"].to_list()])
clusterid, error, nfound = kcluster(matrix, nclusters=num_clusters)
#print(clusterid)

cluster_counts = Counter(clusterid)
sorted_cluster_counts = dict(sorted(cluster_counts.items()))
print(f"Total number of clusters: {num_clusters}")
print("Number of sequences in each cluster:")
for cluster, count in sorted_cluster_counts.items():
    print(f"Cluster {cluster}: {count} sequences")

Total number of clusters: 10
Number of sequences in each cluster:
Cluster 0: 45 sequences
Cluster 1: 3 sequences
Cluster 2: 1 sequences
Cluster 3: 15 sequences
Cluster 4: 41 sequences
Cluster 5: 1 sequences
Cluster 6: 19 sequences
Cluster 7: 1 sequences
Cluster 8: 37 sequences
Cluster 9: 1 sequences


/tmp/ipykernel_1767711/722939898.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  matrix = np.asarray([np.fromstring(s, dtype=np.uint8) for s in filtered["seq"].to_list()])


In [86]:
from Bio.Cluster import kcluster
import numpy as np
from collections import Counter

def one_hot_encode_from_protein_sequence(sequence):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    encoding = np.zeros((len(sequence), len(amino_acids)), dtype=np.uint8)
    for i, aa in enumerate(sequence):
        if aa in amino_acids:
            encoding[i, amino_acids.index(aa)] = 1
    return encoding

num_clusters = 10

sequences = filtered["seq"].to_list()
matrix = np.asarray([one_hot_encode_from_protein_sequence(seq) for seq in sequences])
matrix = matrix.reshape(len(matrix), -1)
clusterid, error, nfound = kcluster(matrix, nclusters=num_clusters)

cluster_counts = Counter(clusterid)
sorted_cluster_counts = dict(sorted(cluster_counts.items()))

print("Number of sequences in each cluster:")
for cluster, count in sorted_cluster_counts.items():
    print(f"Cluster {cluster}: {count} sequences")


Number of sequences in each cluster:
Cluster 0: 27 sequences
Cluster 1: 1 sequences
Cluster 2: 1 sequences
Cluster 3: 40 sequences
Cluster 4: 1 sequences
Cluster 5: 13 sequences
Cluster 6: 44 sequences
Cluster 7: 1 sequences
Cluster 8: 2 sequences
Cluster 9: 34 sequences
